<a href="https://colab.research.google.com/github/manikanta5315/Bayes_Assignments/blob/main/Amazon_Customer_Query_Resolution_MAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiagent system for Customer support & query resolution at Amazon.

In [ ]:
!pip install -q crewai 'crewai[tools]' langchain_groq gradio  -U duckduckgo-search -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import warnings
from crewai import Agent, Task, Crew, LLM
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
import json
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from bs4 import BeautifulSoup
import requests
from typing import Dict, Any
import re
from typing import List



# Warning control
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import userdata

serper_api_key = userdata.get('serperAPI')
os.environ["SERPER_API_KEY"] = serper_api_key

gemini_api_key = userdata.get('geminiAPI')
os.environ["GEMINI_API_KEY"] = gemini_api_key

# Initialize tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()



In [ ]:
# Initialize LLM with environment variable
gem_llm = LLM(model="gemini/gemini-1.5-flash",  api_key=gemini_api_key)

In [ ]:
@tool("get_amazon_product_info")
def get_amazon_product_info(query: str) -> str:
    """
    Search for product information from Amazon using DuckDuckGo search results.

    Args:
        query (str): Search query for the product

    Returns:
        str: JSON formatted string with product information
    """
    try:
        search_tool = DuckDuckGoSearchRun()
        amazon_query = f"{query} site:amazon.com"
        raw_results = search_tool.run(amazon_query)

        def extract_product_info(text: str) -> List[Dict[str, Any]]:
            products = []
            segments = text.split('\n')
            current_product = {}

            for segment in segments:
                price_match = re.search(r'\$[\d,]+\.?\d*', segment)
                title_indicators = ['amazon.com:', 'Amazon.com:', 'Shop at Amazon:']

                if any(indicator in segment for indicator in title_indicators):
                    if current_product.get('title'):
                        products.append(current_product)
                        current_product = {}
                    current_product['title'] = segment.strip().split(':')[-1].strip()

                if price_match and 'title' in current_product:
                    current_product['price'] = price_match.group(0)
                    products.append(current_product)
                    current_product = {}

            return products

        products = extract_product_info(raw_results)

        response = {
            "query": query,
            "total_products_found": len(products),
            "products": products
        }

        return json.dumps(response, indent=2)

    except Exception as e:
        return json.dumps({"error": str(e)}, indent=2)

In [ ]:
# Create Agents
customer_service_agent = Agent(
    role="Customer Service Representative",
    goal="Handle initial customer queries and provide first-level support",
    backstory=(
        "Experienced Amazon customer service representative with knowledge "
        "of policies and procedures. Expert at understanding customer needs."
    ),
    llm=gem_llm,
    tools=[search_tool, get_amazon_product_info],
    verbose=True
)

product_specialist = Agent(
    role="Product Specialist",
    goal="Research and provide detailed product information from Amazon",
    backstory=(
        "Expert in Amazon's product catalog with deep knowledge of various "
        "product categories and features. Specializes in product research and pricing."
    ),
    llm=gem_llm,
    tools=[search_tool, scrape_tool, get_amazon_product_info],
    verbose=True
)

product_availability_agent = Agent(
    role="Product Availability and Size Agent",
    goal="Retrieve product availability and size details from Amazon",
    backstory=(
        "Specialized in tracking product availability and size options on Amazon. "
        "Expert at retrieving real-time inventory data and size information."
    ),
    llm=gem_llm,
    tools=[search_tool, scrape_tool, get_amazon_product_info],
    verbose=True
)

resolution_specialist = Agent(
    role="Resolution Specialist",
    goal="Handle complex issues and provide solutions",
    backstory=(
        "Senior specialist handling complex cases and providing comprehensive "
        "solutions. Expert in Amazon's policies and resolution procedures."
    ),
    llm=gem_llm,
    tools=[search_tool, scrape_tool, get_amazon_product_info],
    verbose=True
)

In [ ]:

initial_analysis_task = Task(
    description=(
        "Analyze the following customer query: '{customer_query}' \n"
        "Identify the type of issue, priority, and key information needed for resolution."
    ),
    expected_output=(
        "A structured analysis document containing:\n"
        "1. Issue classification and priority level\n"
        "2. Key information extracted from the query\n"
        "3. Initial assessment of resolution path\n"
        "4. Required additional information if any"
    ),
    agent=customer_service_agent
)

product_research_task = Task(
    description=(
        "Research and compile detailed product information for the query: '{customer_query}' \n"
        "Focus on information relevant to the customer's specific needs or concerns."
    ),
    expected_output=(
        "A detailed product report containing:\n"
        "1. Complete product specifications and features\n"
        "2. Current pricing and availability status\n"
        "3. Customer review analysis and ratings\n"
        "4. Comparative analysis with similar products if relevant"
    ),
    context=[initial_analysis_task],
    agent=product_specialist
)

availability_check_task = Task(
    description=(
        "Check real-time product availability for: '{customer_query}' \n"
        "Provide detailed inventory status and size availability information."
    ),
    expected_output=(
        "Detailed availability report including:\n"
        "1. Current stock status\n"
        "2. Available sizes and colors\n"
        "3. Expected restock dates if applicable\n"
        "4. Alternative options if unavailable"
    ),
    context=[product_research_task],
    agent=product_availability_agent
)

resolution_task = Task(
    description=(
        "Develop a comprehensive solution for: '{customer_query}' \n"
        "Create step-by-step instructions for resolution based on all gathered information."
    ),
    expected_output=(
        "A complete resolution document containing:\n"
        "1. Step-by-step resolution instructions\n"
        "2. Relevant policy references\n"
        "3. Alternative solutions if applicable\n"
        "4. Follow-up recommendations"
    ),
    context=[initial_analysis_task, product_research_task, availability_check_task],
    agent=resolution_specialist
)


In [ ]:
# Create crew
support_crew = Crew(
    agents=[
        customer_service_agent,
        product_specialist,
        product_availability_agent,
        resolution_specialist
    ],
    tasks=[
        initial_analysis_task,
        product_research_task,
        availability_check_task,
        resolution_task
    ],
    verbose=True
)

In [ ]:



#customer_query = input("Please enter the customer's query: ")

# Define the input structure for customer support query
customer_query = {
    'customer_query': """I ordered a laptop (Order #ABC123456) three days ago with next-day delivery,
    but it hasn't arrived yet. The tracking hasn't updated in 48 hours.
    This is urgent as I need it for work.""",

    'order_details': {
        'order_number': 'ABC123456',
        'order_date': '2024-12-19',
        'expected_delivery': '2024-12-20',
        'product_category': 'Electronics',
        'shipping_method': 'Next-Day Delivery'
    },

    'customer_info': {
        'priority_status': 'Prime',
        'previous_cases': 'None'
    }
}





print("-" * 50)
result = support_crew.kickoff({"customer_query": customer_query})
print("\nResponse:")
print(result)

--------------------------------------------------
# Agent: Customer Service Representative
## Task: Analyze the following customer query: '{'customer_query': "I ordered a laptop (Order #ABC123456) three days ago with next-day delivery,\n    but it hasn't arrived yet. The tracking hasn't updated in 48 hours.\n    This is urgent as I need it for work.", 'order_details': {'order_number': 'ABC123456', 'order_date': '2024-12-19', 'expected_delivery': '2024-12-20', 'product_category': 'Electronics', 'shipping_method': 'Next-Day Delivery'}, 'customer_info': {'priority_status': 'Prime', 'previous_cases': 'None'}}' 
Identify the type of issue, priority, and key information needed for resolution.


# Agent: Customer Service Representative
## Thought: Thought:I need to analyze the customer query to identify the issue, priority, and necessary information for resolution.  The customer's order hasn't arrived despite selecting next-day delivery.
## Using tool: Search the internet
## Tool Input: 
"{\